This function performs the orthogonal projection of the vector "gradient" over the hyperplane constructed from vector "y" and the fact that we are currently located at certain sides of the constraint cube 0 <= alpha_i <= C. This function will be used later on.

In [ ]:
def orthogonal_projection(gradient, y, sides):
    import sys
    import numpy as np

    m = y.size
    if (len(sides) > m):
        sys.exit("Error: len(sides) > m")

    # We normalize y. We use a vector y equal to zeros to simulate no hyperplane
    norm = np.linalg.norm(y)
    y_normalized = y.copy()
    if not (norm == 0):
        y_normalized = y_normalized / np.linalg.norm(y)

    # We now apply the Gram-Schmidt algorithm to construct the
    # projection subspace S
    c_ortho = np.zeros((len(sides),m))
    for i in range(len(sides)):
        c_ortho[i,sides[i]] = 1.0;
        c_ortho[i,:] = c_ortho[i,:] - np.dot(c_ortho[i,:], y_normalized) * y_normalized
        for j in range(0,i):
            c_ortho[i,:] = c_ortho[i,:] - np.dot(c_ortho[i,:], c_ortho[j,:]) * c_ortho[j,:]
        c_ortho[i,:] = c_ortho[i,:] / np.linalg.norm(c_ortho[i,:])
        
    # This is the gradient projection orthogonal to the subspace S constructed
    # previously
    g_project = gradient.copy()
    g_project = g_project - np.dot(g_project, y_normalized) * y_normalized
    for i in range(len(sides)):
        g_project = g_project - np.dot(g_project, c_ortho[i,:]) * c_ortho[i,:]  
        
    return g_project

We begin with an example. Compute the values of a quadratic form and plot it. We also plot the restrition cube.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# These are the constraints. Please do not set to a high value for the moment.
C=3.0

# Let us construct a quadratic function 0.5 * x^T * A * x - h^T * x
A = np.identity(2)
A[1,1]=20.0
h = np.array([4, 80])

# Let us plot the function. Observe that "x1" is horizontal in the plot,
# whereas "x2" is respectively vertical.
x1 = np.arange(-0.2, 10.0, 0.2)
x2 = np.arange(-0.2, 7.0, 0.2)

X1, X2 = np.meshgrid(x1, x2)

Z = np.zeros((x2.size, x1.size))

for i in range(x2.size):
    for j in range(x1.size):
        x_tmp = np.array([x1[j],x2[i]])
        z_tmp = 0.5 * np.dot(x_tmp,np.dot(A,x_tmp)) - np.dot(h,x_tmp)
        Z[i,j] = z_tmp
        
fig = plt.figure()
plt.contour(X1, X2, Z, 10)

# Plot the restriction cube
plt.plot([0, 0, C, C, 0], [0, C, C, 0, 0])

We now perform a test with the computation of the optimum value of beta, the search step. Here you may test with the initial point and a search direction. The search direction does not have to be a descent direction. If it is not a descent direction, the value of beta will be negative.

In [ ]:
# Initial point
alpha_0 = np.array([0.5,0.5])

# The desent direction. First the x1 component, then the x2 component
# Try here different values!
descent = np.array([1.0,0.5])
                     
# Let us compute the maximum step that we may take. The equation
# is given in the lab document
p = descent.copy()
beta_opt_denominator = np.dot(p,np.dot(A,p))
beta_opt_numerator   = np.dot(p,h - np.dot(A,alpha_0))
beta_opt = (beta_opt_numerator / beta_opt_denominator)

print "The optimum value of beta is", beta_opt

# This is the new point
alpha_1 = alpha_0 + beta_opt * p

# Plot it!!!
fig = plt.figure()
plt.contour(X1, X2, Z, 10)
plt.plot([alpha_0[0], alpha_1[0]], [alpha_0[1], alpha_1[1]])

This is an example in which we show how to perform a gradient descent restricted to a particular cube. Here no hyperplane "y" is constructed. We simulate the latter effect by setting y = np.array([0, 0]) (see function orthogonal_projection since the code has been adapted to take this case into account).

In [ ]:
# Initial point. The initial point should be inside the cube
alpha = np.array([0.5,0.5])

# This are the restictions of the cube. Currently no restrictions since
# we are inside the cube
sides = []

# We initialize the effect of no hyperplane (see the code orthogonal_projection)
y = np.array([0, 0])
m = y.size

# We plot the function and the restriction cube
fig = plt.figure()
plt.contour(X1, X2, Z, 10)
plt.plot([0, 0, C, C, 0], [0, C, C, 0, 0])

# Iterations
cont = 0
while (cont < 100):
    cont=cont+1
    print "Iteration", cont
    
    # Gradient of the quadratic form
    gradient = np.dot(A, alpha) - h
    descent  = - gradient.copy()
    
    # This is the search direction, that is, the gradient projected taking into 
    # account the restriction of the cube
    p = orthogonal_projection(descent, y, sides)
    
    # We manually set to zero the search direction if we are on side "i"
    for i in sides:
        p[i] = 0.0
        
    print "   Descent is", p
        
    # Compute the norm of p. Just to be sure that we can move. 
    norm = np.linalg.norm(p)
    if (norm < 1e-10):
        print "Norm of p very small! Finished!"
        break

    # Compute the optimum value of beta, the search step
    beta_opt_denominator = np.dot(p, np.dot(A,p))
    beta_opt_numerator   = np.dot(p, h - np.dot(A,alpha)) 
    beta_opt = (beta_opt_numerator / beta_opt_denominator)
    
    print "   beta_opt is", beta_opt
    
    # Which is the maximum value of beta that can be taken?
    # Since the function to minimize is quadratic we have to add 
    # new elements to sides (i.e. no elements are removed from sides)
    beta_max = beta_opt
    for i in range(m):
        if not i in sides:
            if (p[i] > 0):    # We move to the upper face of the restriction
                beta_tmp = (C - alpha[i]) / p[i]
            elif (p[i] < 0):  # We move to the lower face of the restriction
                beta_tmp = ( - alpha[i] ) / p[i]
                
            if (beta_tmp < beta_max):
                beta_max = beta_tmp
                sides.extend([i])
                
    print "   beta_max is", beta_max
                
    # Update the value of alpha            
    alpha_new = alpha + beta_max * p
    
    # Plot
    plt.plot([alpha[0], alpha_new[0]], [alpha[1], alpha_new[1]], 'r')
    
    # Update
    alpha = alpha_new
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Try this first!!! We have one sample for each class!!!
c1 = np.array([[0.5, 0.3]])
c2 = np.array([[5.0, 6.0]])

# Once the previous works, try with this
# Generate random data. The next code is of Oriols' notebook.
#m1 = [0.,0.]
#s1 = [[1,-0.9],[-0.9,1]]
#m2 = [3.,6.]
#s2 = [[1,0],[0,1]]
#c1 = np.random.multivariate_normal(m1,s1,2)
#c2 = np.random.multivariate_normal(m2,s2,1)

# Perform the plot
fig = plt.figure()
plt.plot(c1[:,0],c1[:,1],'r.')
plt.plot(c2[:,0],c2[:,1],'b.')
plt.xlim([-10, 10])
plt.ylim([-10, 10])

# We concatenate the data of c1 and c2. Take into account that variable
# "y" is our hyperplane. 
x = np.r_[c1,c2]
y = np.r_[-np.ones(c1.shape[0]),np.ones(c2.shape[0])]

# Let us now construct matrices X and Y as explained in the lab document.
m = y.size
print "The problem has", m, "training elements"

# This is the restriction cube. You may use different values of C.
C = 100000.0  
sides = []
               
# Our initial values for alpha. The initial point has to be over the
# hyperplane
alpha = np.empty(m)
alpha.fill(0.0)    
               
# Construct matrices A and h in order to be able to use the code
# from this notebook! We are going to minimize the quadratic function 
# f(alpha) = 0.5 * alpha^T * A * alpha - h^T * alpha 

# TODO TODO: PUT YOUR CODE HERE!!!

# Perform now the iterations. 
               
# TODO TODO: PUT YOUR CODE HERE!!!
               
# Once iterations are finished, recover the value of w and b.
# The equations are indicated in the document.
               
# TODO TODO: PUT YOUR CODE HERE!!!
               
# Some plot would be fine
               
# TODO TODO: PUT YOUR CODE HERE!!!